In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Layer , Dense, Conv2D, MaxPooling2D , Activation

In [8]:
# Creating the Local Response Normalization function from the paper
class ResponseNormalization(Layer):
    def __init__(self, k=2, n=5, epsilon=1e-4, beta=0.75, **kwargs):
        super(ResponseNormalization, self).__init__(**kwargs)
        self.k = k
        self.n = n
        self.epsilon = epsilon
        self.beta = beta

    def call(self, inputs):
        # Compute squared activations
        squared_activations = tf.square(inputs)

        # Compute sum of squared activations for adjacent kernel maps
        sum_squared_activations = tf.nn.avg_pool(
            squared_activations,
            ksize=[1, self.n, self.n, 1],
            strides=[1, 1, 1, 1],
            padding='SAME'
        )

        # Apply normalization formula
        normalized_output = inputs / tf.pow(self.k + self.epsilon * sum_squared_activations, self.beta)

        return normalized_output

In [9]:
# Declaring the ReLu function
relu = tf.keras.activations.relu
# Declaring the normalizer
normalizer = ResponseNormalization()

In [10]:
def create_alexnet_model(input_shape=(224, 224, 3)):
    inputs = tf.keras.Input(shape=input_shape, name="input")
    x = Conv2D(96, (11, 11), strides=(4, 4), activation='relu', name='conv1')(inputs)
    x = relu(x)
    x = normalizer(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = Conv2D(256, (5, 5), strides=(1, 1), activation='relu', name='conv2')(x)
    x = relu(x)
    x = normalizer(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = Conv2D(384, (3, 3), strides=(1, 1), activation='relu', name='conv3')(x)
    x = relu(x)
    x = Conv2D(384, (3, 3), strides=(1, 1), activation='relu', name='conv4')(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), activation='relu', name='conv5')(x)
    x = relu(x)
    x = normalizer(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)
    outputs = Dense(1000, activation='softmax')(x)
    # Create and return the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="alexnet")
    return model

In [13]:
model = create_alexnet_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.summary()

Model: "alexnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1 (Conv2D)              (None, 54, 54, 96)           34944     ['input[0][0]']               
                                                                                                  
 tf.nn.relu_4 (TFOpLambda)   (None, 54, 54, 96)           0         ['conv1[0][0]']               
                                                                                                  
 response_normalization (Re  multiple                     0         ['tf.nn.relu_4[0][0]',        
 sponseNormalization)                                                'tf.nn.relu_5[0][0]',  